# Spreedly.ai - API + LLM Demo

This notebook is "inspired" by 🦜⛓️'s OpenAPI [agents](https://python.langchain.com/en/latest/modules/agents/toolkits/examples/openapi.html) and [chains](https://github.com/hwchase17/langchain/blob/master/docs/modules/chains/examples/openapi.ipynb) examples.

## Requirements

- Ensure you have selected the Jupyter-provided `venv`
- Remember to run `python3 -m venv venv`
- Create a `.env` with the following format:
```
SPREEDLY_OPEN_API_SPEC_YAML_LOC=path/to/spec.yaml
SPREEDLY_ACCESS_TOKEN=<base64 encoded user:pass>
OPENAI_API_KEY=<your OPEN AI API KEY>
```

## Load Spreedly API Spec

In [1]:
import os, yaml
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec

yaml_location =  os.getenv('SPREEDLY_OPEN_API_SPEC_YAML_LOC')
with open(yaml_location) as f:
    raw_spreedly_openai_api_spec = yaml.load(f, Loader=yaml.Loader)

spreedly_openai_api_spec = reduce_openapi_spec(raw_spreedly_openai_api_spec)

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [2]:
from langchain.requests import RequestsWrapper

access_token = os.getenv('SPREEDLY_ACCESS_TOKEN')
headers = {
        'Authorization': f'Basic {access_token}'
    }
requests_wrapper = RequestsWrapper(headers=headers)

Kick the tires, can we instrospect the API?

In [ ]:
import json

endpoints = [
    (route, operation)
    for route, operations in raw_spreedly_openai_api_spec["paths"].items()
    for operation in operations
    if operation in ["get", "post"]
]

display(f"There are {len(endpoints)} endpoints available")

display(f"""Server Base URL: {raw_spreedly_openai_api_spec["servers"][0]["url"]}""")

: 

Prepare our objects

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits.openapi import planner


# relies on OPENAI_API_KEY from .env file.
llm = ChatOpenAI(model_name="gpt-4", temperature=0.0)

: 

Create Agent

In [ ]:
spreedly_agent = planner.create_openapi_agent(spreedly_openai_api_spec, requests_wrapper, llm, verbose=True)

: 

## List Supported Gateways

In [ ]:
user_query = "List all gateways supported by Spreedly"

spreedly_agent.run(user_query)

: 

## Create a gateway (sandbox mode)

 Some glue code first

In [ ]:
%%script false --no-raise-error
## Currently disabled.

import os

adyen_creds = f"""{{
    "ADYEN_USERNAME": {os.getenv('ADYEN_USERNAME')},
    "ADYEN_PASSWORD": {os.getenv('ADYEN_PASSWORD')},
    "ADYEN_MERCHANT_ACCOUNT": {os.getenv('ADYEN_MERCHANT_ACCOUNT')}
}}
"""


: 

Send the query

In [ ]:
%%script false --no-raise-error
## Currently disabled.

spreedly_agent = planner.create_openapi_agent(spreedly_openai_api_spec, requests_wrapper, llm, verbose=True)

user_query ="Create a new adyen sandbox gateway using these gateway credentials: {adyen_creds}"

spreedly_agent.run(user_query)

: 